Joint Shapley value calculations on the Rotten Tomatoes dataset (Pang and Lee, 2005, PACL)

In [1]:
import copy
import random
import scipy

import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer

from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.
C:\ProgramData\Anaconda3\envs\Shapley-values\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\ProgramData\Anaconda3\envs\Shapley-values\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\ProgramData\Anaconda3\envs\Shapley-values\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\ProgramData\Anaco

## Set parameters

In [2]:
k = 2
take_x_most_common_words = 1000
num_sampling_iterations = 5
np.random.seed(10) # Set random seed for consistency

In [3]:
# 5331 pos, 5331 neg reviews; the lines here refer to lines in the pos and neg review files separately
train_lines = list(range(0,199))
train_lines.extend(range(200,1551))
train_lines.extend(range(1600,5331))

test_lines = list(range(1551, 1600))
test_lines.insert(0,199)

## Load data
Review variable names suffixed with `_ss` are a list of strings, with one string representing an entire review. 

Review variable names suffixed with `_ls` are a list of list of strings, with one string representing an entire review. 

Review variable names suffixed with `_vc` are lists of lists of integers, with a list of integers representing an entire review, and an integer representing a word.

Review variable names suffixed with `_oh` are one-hot encoded pandas Dataframes of reviews, with the column names representing the numeric representation of a word.

### Obtain Rotten tomatoes data

In [4]:
def get_rt_data(fname, line_range):
    with open(fname, encoding='utf-8', errors='ignore') as file:
        all_lines = list(file)
        as_single_string = [all_lines[line_num] for line_num in line_range]
        as_list_of_strings = [line.strip().split() for line in as_single_string]
    return as_single_string, as_list_of_strings

#### Manual pre-processing converts "a" -> "aaaa" and "won't" -> "wont" to ease tokenizing

In [5]:
any_y = np.concatenate((np.ones(len(train_lines),dtype=bool), np.zeros(len(train_lines), dtype=bool)))

train_pos_reviews_ss, train_pos_reviews_ls = get_rt_data("../datasets/rt-polaritydata/rt-polarity-preprocessed.pos", train_lines) 
train_neg_reviews_ss, train_neg_reviews_ls = get_rt_data("../datasets/rt-polaritydata/rt-polarity-preprocessed.neg", train_lines)
train_reviews_ss = train_pos_reviews_ss + train_neg_reviews_ss 
train_reviews_ls = train_pos_reviews_ls + train_neg_reviews_ls
train_y = np.concatenate(
    (np.ones(len(train_lines),dtype=bool), 
     np.zeros(len(train_lines), dtype=bool))
)

test_pos_reviews_ss, test_pos_reviews_ls = get_rt_data("../datasets/rt-polaritydata/rt-polarity-preprocessed.pos", test_lines) 
test_neg_reviews_ss, test_neg_reviews_ls = get_rt_data("../datasets/rt-polaritydata/rt-polarity-preprocessed.neg", test_lines) 
test_reviews_ss = test_pos_reviews_ss + test_neg_reviews_ss 
test_reviews_ls = test_pos_reviews_ls + test_neg_reviews_ls
test_y = np.concatenate(
    (np.ones(len(test_lines),dtype=bool), 
     np.zeros(len(test_lines), dtype=bool))
)

all_reviews_ss = train_reviews_ss + test_reviews_ss
all_reviews_ls = train_reviews_ls + test_reviews_ls

In [6]:
print(f"Train set has {len(train_reviews_ss)} reviews; example review:\n{train_reviews_ls[0]}")

Train set has 10562 reviews; example review:
['the', 'rock', 'is', 'destined', 'to', 'be', 'the', '21st', "century's", 'new', '"', 'conan', '"', 'and', 'that', "he's", 'going', 'to', 'make', 'aaaa', 'splash', 'even', 'greater', 'than', 'arnold', 'schwarzenegger', ',', 'jean-claud', 'van', 'damme', 'or', 'steven', 'segal', '.']


In [7]:
print(f"Test set has {len(test_reviews_ss)} reviews; example review:\n{test_reviews_ls[0]}")

Test set has 100 reviews; example review:
['australian', 'actor/director', 'john', 'polson', 'and', 'award-winning', 'english', 'cinematographer', 'giles', 'nuttgens', 'make', 'aaaa', 'terrific', 'effort', 'at', 'disguising', 'the', 'obvious', 'with', 'energy', 'and', 'innovation', '.']


## Add hand-picked example n-grams terms from existing literature
#### Socher et al. (EMNLP, 2013), Dhamdhere et al. (ICML, 2020).

In [8]:
target_vocab_DSA = dict(
    t2dsa = ["wont disappointed", "both inspiring", "aaaa crisp", "excellent youthful", "john terrific"],
)

target_vocab = dict(
    t1 = ['engaging', 'best', 'powerful', 'love', 'beautiful', 'bad', 'dull', 
          'boring', 'fails', 'worst', 'stupid', 'painfully'],
    t2 = ['excellent performances', 'aaaa masterpiece', 'masterful film', 'wonderful movie',
          'marvelous performances', 'worst movie', 'very bad', 'shapeless mess',
          'worst thing', 'instantly forgettable', 'complete failure'],
    t3 = ['an amazing performance', 'wonderful all ages triumph', 'aaaa wonderful movie',
          'most visually stunning', 'for worst movie', 'aaaa lousy movie', 'aaaa complete failure',
          'most painfully marginal', 'very bad sign'],
    t4 = ['gorgeous imagery effective performances', 'just another bad movie',
          'aaaa humorless disjointed mess'],
    t5 = ['nicely acted and beautifully shot', 'the best of the year',
           'aaaa terrific american sports movie', 'refreshingly honest and ultimately touching',
           'silliest and most incoherent movie', 'completely crass and forgettable movie',
           'aaaa cumbersome and cliche ridden movie'],
    t6 = ['aaaa trashy exploitative thoroughly unpleasant experience'],
    t7 = ['aaaa masterful film from aaaa master filmmaker', 'this sloppy drama is an empty vessel',
          'quickly drags on becoming boring and predictable'],
    t8 = ['one of the best films of the year', 'aaaa love for films shines through each frame',
          'created aaaa masterful piece of artistry right here',
          'be the worst special effects creation of the year'],
    t2dsa = ["wont disappointed", "both inspiring", "aaaa crisp", "excellent youthful", "john terrific"],
)

def target_vocab_to_reviews_ss(tv):
    return [' '.join(v) for v in tv.values()]

def target_vocab_DSA_to_reviews_ss(tv):
    return [' '.join(v) for v in tv.values()]

## Manage the formats of the reviews

#### We add some helper functions to convert between our chosen review formats

In [9]:
def ss_to_oh(reviews_ss, vectorizer):
    oh_reviews = vectorizer.transform(reviews_ss).toarray().astype(bool)
    return pd.DataFrame(
        columns = vectorizer.get_feature_names(), data=oh_reviews
    )    

def ss_to_vc_old(reviews_ss, vectorizer):
    return [
        [vectorizer.get_feature_names().index(w) for w in rev_ss.split()]
        for rev_ss in reviews_ss
    ]

def ss_to_vc(reviews_ss, vectorizer):
    as_oh = ss_to_oh(reviews_ss, vectorizer)
    return [
        [i for i, bl in enumerate(as_oh_row) if bl]
        for as_oh_row in as_oh.values
    ]

#### A 'vectorizer' is used to encode reviews numerically

In [10]:
def get_vectorizer(reviews_ss):
    vectorizer = CountVectorizer()
    return vectorizer.fit(reviews_ss)

all_vectorizer = get_vectorizer(
    all_reviews_ss + target_vocab_to_reviews_ss(target_vocab)
)

num_features = len(all_vectorizer.get_feature_names())
print(f"Total vocabulary {num_features}")

Total vocabulary 18326


#### A function to take the top n most common words from our chosen body of work

In [11]:
def get_top_n_words(corpus, n):
    vec = CountVectorizer().fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

#### Get the top x words and create a vectorizer of these plus the DSA words

In [12]:
top_n_words_ss = [
    word for word, _ in get_top_n_words(all_reviews_ss, take_x_most_common_words)
]
    
DSA_vectorizer = get_vectorizer(
    top_n_words_ss + target_vocab_DSA_to_reviews_ss(target_vocab_DSA)
)

num_DSA_features = len(DSA_vectorizer.get_feature_names())

print(f"Total vocabulary {num_DSA_features}")

Total vocabulary 1004


## Shuffle the training set

In [13]:
def get_shuffle_index(x_len):
    return np.random.permutation(np.arange(x_len))   

In [14]:
si = get_shuffle_index(len(train_reviews_ss))
ss_train = [train_reviews_ss[i] for i in si]
train_y = train_y[si]

#### One-hot encode the (now shuffled) training set

In [15]:
oh_train = ss_to_oh(ss_train, DSA_vectorizer)

#### One-hot encode the test set (no shuffling is needed)

In [16]:
oh_test = ss_to_oh(test_reviews_ss, DSA_vectorizer)

## Define the ML model

In [17]:
model = Sequential()
model.add(Dense(16, input_dim=oh_train.shape[1], activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

#### Train on one-hot encoded data

In [18]:
model.compile(optimizer="rmsprop", loss="binary_crossentropy", metrics=["accuracy"])
model.fit(oh_train.values, train_y, epochs=4, batch_size=5)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/4
10562/10562 [==============================] - 3s 272us/step - loss: 0.6252 - accuracy: 0.6415
Epoch 2/4
10562/10562 [==============================] - 3s 238us/step - loss: 0.5349 - accuracy: 0.7343
Epoch 3/4
10562/10562 [==============================] - 2s 227us/step - loss: 0.5112 - accuracy: 0.7551
Epoch 4/4
10562/10562 [==============================] - 2s 229us/step - loss: 0.4997 - accuracy: 0.7627


In [19]:
_, accuracy = model.evaluate(oh_test.values, test_y)
print(f"Model accuracy on test set is {accuracy:.2%}")

100/100 [==============================] - 0s 279us/step
Model accuracy on test set is 80.00%


#### Some examples of incorrect predictions

In [20]:
predictions = (model.predict(oh_test.values) > 0.5).astype(np.int32)
incorrect_predictions = [
    (rev, predicted[0], actual) 
    for (rev, predicted, actual) in zip(test_reviews_ss, predictions, test_y)
    if predicted[0] != int(actual)
]
for rev, predicted, actual in incorrect_predictions[:3]:
    print(f"{rev}")
    print(f"Predicted {predicted}, actual {int(actual)}\n\n")

this is wild surreal stuff , but brilliant and the camera just kind of sits there and lets you look at this and its like you're going from one room to the next and none of them have any relation to the other . 

Predicted 0, actual 1


it's aaaa demented kitsch mess ( although the smeary digital video does match the muddled narrative ) , but it's savvy about celebrity and has more guts and energy than much of what will open this year . 

Predicted 0, actual 1


aaaa welcome relief from baseball movies that try too hard to be mythic , this one is aaaa sweet and modest and ultimately winning story . 

Predicted 0, actual 1




#### Helper functions to generate powerset of features, up to maximum set size of k

In [21]:
# includes the emptyset; don't call this directly as yields index out of bounds
def get_powerset_to_k(seq, k, init=True):
    if init:
        seq = [s if type(s) == frozenset else frozenset([s]) for s in seq]
    if len(seq) <= 1:
        yield seq[0]
        yield frozenset()
    else: 
        for item in get_powerset_to_k(seq[1:], k, False):
            if len(item) <= k - 1:
                yield seq[0].union(item)
            yield item

In [22]:
# removes the empty set: can call this directly
def get_powerset_to_k_ex_emptyset(seq, k):
    gen = get_powerset_to_k(seq, k)
    for item in gen:
        if item != frozenset():
            yield item

In [22]:
def choose(n, k):
    return scipy.special.comb(n, k)

#### The q values allow us to probability-weight our sampled results

In [23]:
def get_q_values_for_k(k, num_local_features):
    q = np.zeros(num_local_features, dtype=np.longdouble) 
    q0den = sum([
        choose(num_local_features, s) for s in range(1, k+1)
    ])
    q[0] = 1/q0den 
    for r in range(1, num_local_features):
        limd = min(k, (num_local_features - r))
        limn = max((r - k), 0)
        qden = sum([
            choose(num_local_features - r, s) for s in range(1, limd+1)
        ])
        qnum = sum([
            choose(r, s) * q[s] for s in range(limn, r)
        ])
        q[r] = qnum / qden
    # Check that the checksum is satisfied
    checksum = sum([
        choose(num_local_features, i) * q[i] 
        for i in range((num_local_features - k), num_local_features)
    ])
    assert(np.isclose(checksum, 1.0))
    return q

_all_q_values = [get_q_values_for_k(_k, num_DSA_features) for _k in range(1, k+1)]
q_values = np.vstack(_all_q_values)

### Take a single sample review

In [24]:
sample_review_oh = oh_test.iloc[0]
print(f'sample review: {test_reviews_ss[0]}')
all_features_set = set(range(1, num_DSA_features + 1))

sample review: australian actor/director john polson and award-winning english cinematographer giles nuttgens make aaaa terrific effort at disguising the obvious with energy and innovation . 



## The Joint Shapley calculation function

In [25]:
def calculate_phi_t(
    review_oh,
    value_function,
    coalition_vc,
    num_features,
    q_values,
    num_sampling_iterations,
):
    coalition_size = len(coalition_vc)
    p_values = np.zeros((num_features - coalition_size + 1), dtype=np.longdouble)
    for i in range(1, num_features - coalition_size + 2):
        q_value = q_values[k-1, i-1]
        num_ft_choose_len = choose((num_features - coalition_size), (i - 1))
        p_values[i - 1] = num_ft_choose_len * q_value
    hat_p_values = p_values / np.sum(p_values)
    estimates = np.zeros(num_sampling_iterations)

    for itr in range(0, num_sampling_iterations):
        U = np.random.uniform(0, 1, 1)
        X = sum(U > hat_p_values.cumsum())
        S = set()
        for w in range(1, X + 1):
            possible_next = set(range(1, num_features + 1)).difference(
                set(coalition_vc).union(S))
            r = random.sample(possible_next, 1)
            S = S.union(set(r))

        z = np.random.randint(0, 2, num_features)
        xminusT = np.copy(review_oh)
        xplusT = np.copy(review_oh)
        for h in set(range(0, num_features)).difference(S):
            xminusT[h] = z[h]
        for h in set(range(0, num_features)).difference(S.union(set(coalition_vc))):
            xplusT[h] = z[h]

        estimates[itr] = value_function(xplusT) - value_function(xminusT)
    return np.mean(estimates) * np.sum(p_values)

#### We use a 'value function' to give us the value of a given coalition; in this case the 'value' is the prediction it receives from our ML model

In [26]:
value_f = lambda oh: model.predict(oh[None, :])[0][0]

#### We look at our target vocabulary from Dhamdhere et al. (2020)

In [27]:
# Obtain bigrams used in Dhamdhere et al as strings
t2dsa = target_vocab["t2dsa"]
# Vectorize bigrams
t2dsa_vc = ss_to_vc(t2dsa, DSA_vectorizer)
# Add both bigrams and single words from bigrams to list of coalitions
DSA_coalitions = [cln for cln in t2dsa_vc] + [[word] for cln in t2dsa_vc for word in cln]

#### We can now calculate Joint Shapley values for the Dhamdhere et al. coalitions

In [28]:
test_reviews_vc = ss_to_vc(test_reviews_ss, DSA_vectorizer)
num_review = 38 # sets the review to be analysed
sample_review_oh = oh_test.iloc[num_review]
print(f'test_reviews_vc[num_review] = {test_reviews_vc[num_review]}')

num_times_to_loop = 60

with open("results/250528-review-38-convergence.txt", "a") as a_writer:
    a_writer.write(f'test_reviews_vc[num_review] = {test_reviews_vc[num_review]}\n')

    for loop_num in range(num_times_to_loop+1):
        a_writer.write(f'loop number = {loop_num}\n')
        for coalition in DSA_coalitions:
            js = calculate_phi_t(
                sample_review_oh,
                value_f,
                coalition,
                num_DSA_features,
                q_values,
                num_sampling_iterations
            )
            a_writer.write(f'k = {k}, coalition {coalition}: local jShapley = {js}\n')


test_reviews_vc[num_review] = [44, 101, 434, 447, 677, 860, 946]


#### Iterate through all the reviews in the test set, calculating local joint Shapleys for the DSA features

#### (to reproduce Table 4: set k=3 and iterate through reviews 0, 11, 33, 38, 46)

#### (global joint Shapleys are calculated externally according to the instructions in the paper)

#### (to reproduce Table 5: set k=1 and iterate over all reviews)

In [29]:
for n, review_oh in enumerate(oh_test.iloc):
    for coalition in DSA_coalitions:
        js = calculate_phi_t(
            review_oh,
            value_f,
            coalition,
            num_DSA_features,
            q_values,
            num_sampling_iterations
        )
        print(f'k = {k}, review {n}, coalition {coalition}: local jShapley = {js}')

k = 2, review 0, coalition [209, 978]: local jShapley = -2.099510518642003e-06
k = 2, review 0, coalition [101, 434]: local jShapley = -1.4886589393610838e-05
k = 2, review 0, coalition [3, 174]: local jShapley = 2.3190096850864185e-07
k = 2, review 0, coalition [269, 1003]: local jShapley = -3.188376419995403e-05
k = 2, review 0, coalition [455, 848]: local jShapley = -7.489815944933572e-05
k = 2, review 0, coalition [209]: local jShapley = 4.38324461538257e-05
k = 2, review 0, coalition [978]: local jShapley = -0.0003035084371104904
k = 2, review 0, coalition [101]: local jShapley = 0.0
k = 2, review 0, coalition [434]: local jShapley = -3.947177570884824e-05
k = 2, review 0, coalition [3]: local jShapley = 7.122016502769221e-07
k = 2, review 0, coalition [174]: local jShapley = 3.6732320785569093e-06
k = 2, review 0, coalition [269]: local jShapley = -0.000374570609843023
k = 2, review 0, coalition [1003]: local jShapley = -7.199457261767925e-05
k = 2, review 0, coalition [455]: loc

k = 2, review 7, coalition [209]: local jShapley = 1.6017964792685445e-06
k = 2, review 7, coalition [978]: local jShapley = -3.1545522804273285e-06
k = 2, review 7, coalition [101]: local jShapley = -0.0005801260193943258
k = 2, review 7, coalition [434]: local jShapley = -6.445224425660184e-06
k = 2, review 7, coalition [3]: local jShapley = 7.410733031878358e-05
k = 2, review 7, coalition [174]: local jShapley = 3.1315244837497016e-06
k = 2, review 7, coalition [269]: local jShapley = -5.897845169819412e-06
k = 2, review 7, coalition [1003]: local jShapley = -1.2178889933981704e-05
k = 2, review 7, coalition [455]: local jShapley = 1.3376761801897506e-05
k = 2, review 7, coalition [848]: local jShapley = -4.166129273221132e-05
k = 2, review 8, coalition [209, 978]: local jShapley = -3.196710196095557e-06
k = 2, review 8, coalition [101, 434]: local jShapley = -4.837524617011381e-05
k = 2, review 8, coalition [3, 174]: local jShapley = 7.713796671008894e-06
k = 2, review 8, coalition

k = 2, review 14, coalition [434]: local jShapley = -6.250967903057234e-05
k = 2, review 14, coalition [3]: local jShapley = -5.9626641530601396e-05
k = 2, review 14, coalition [174]: local jShapley = 2.7803932284838163e-07
k = 2, review 14, coalition [269]: local jShapley = -0.0004351753357154726
k = 2, review 14, coalition [1003]: local jShapley = -8.895789351284269e-05
k = 2, review 14, coalition [455]: local jShapley = 0.0001481303000102014
k = 2, review 14, coalition [848]: local jShapley = -6.442924844297515e-05
k = 2, review 15, coalition [209, 978]: local jShapley = 8.416032959450946e-06
k = 2, review 15, coalition [101, 434]: local jShapley = -1.9035274109829083e-05
k = 2, review 15, coalition [3, 174]: local jShapley = 1.676962079341442e-05
k = 2, review 15, coalition [269, 1003]: local jShapley = -2.089443968896513e-06
k = 2, review 15, coalition [455, 848]: local jShapley = -8.289850743516578e-08
k = 2, review 15, coalition [209]: local jShapley = 3.507585461122515e-05
k = 

k = 2, review 21, coalition [269]: local jShapley = -8.047532633743787e-05
k = 2, review 21, coalition [1003]: local jShapley = -2.53074632605759e-05
k = 2, review 21, coalition [455]: local jShapley = 0.0003042784737049071
k = 2, review 21, coalition [848]: local jShapley = -0.00027750123999934775
k = 2, review 22, coalition [209, 978]: local jShapley = -6.482261294266374e-06
k = 2, review 22, coalition [101, 434]: local jShapley = -2.894704504959697e-05
k = 2, review 22, coalition [3, 174]: local jShapley = -7.14916366369759e-06
k = 2, review 22, coalition [269, 1003]: local jShapley = -5.472473485036204e-05
k = 2, review 22, coalition [455, 848]: local jShapley = -4.4551273325984446e-05
k = 2, review 22, coalition [209]: local jShapley = 0.00022777525039605423
k = 2, review 22, coalition [978]: local jShapley = -0.00010285771570808917
k = 2, review 22, coalition [101]: local jShapley = -0.0002982370672804299
k = 2, review 22, coalition [434]: local jShapley = -2.3414132193456515e-05

k = 2, review 28, coalition [848]: local jShapley = -0.0005122099680767297
k = 2, review 29, coalition [209, 978]: local jShapley = 1.1419326880710337e-05
k = 2, review 29, coalition [101, 434]: local jShapley = -2.8252698121947926e-06
k = 2, review 29, coalition [3, 174]: local jShapley = 1.4867712512648826e-06
k = 2, review 29, coalition [269, 1003]: local jShapley = -4.466326390881589e-09
k = 2, review 29, coalition [455, 848]: local jShapley = 4.6356729918208356e-05
k = 2, review 29, coalition [209]: local jShapley = 7.73831303453288e-05
k = 2, review 29, coalition [978]: local jShapley = -0.00023263612243147998
k = 2, review 29, coalition [101]: local jShapley = -7.59511199315073e-06
k = 2, review 29, coalition [434]: local jShapley = -2.3919123745518456e-05
k = 2, review 29, coalition [3]: local jShapley = 6.830820616620252e-05
k = 2, review 29, coalition [174]: local jShapley = 1.163426122052187e-06
k = 2, review 29, coalition [269]: local jShapley = -0.00034584906250464937
k = 

...

k = 2, review 65, coalition [3, 174]: local jShapley = 7.610491584181356e-07
k = 2, review 65, coalition [269, 1003]: local jShapley = -5.4799016993027314e-06
k = 2, review 65, coalition [455, 848]: local jShapley = -8.981103369338818e-06
k = 2, review 65, coalition [209]: local jShapley = 4.420650392610789e-05
k = 2, review 65, coalition [978]: local jShapley = -0.00011385565280810293
k = 2, review 65, coalition [101]: local jShapley = -0.0006518246261891042
k = 2, review 65, coalition [434]: local jShapley = -1.0579753990104791e-07
k = 2, review 65, coalition [3]: local jShapley = 0.00016905383581725713
k = 2, review 65, coalition [174]: local jShapley = 4.487460587298028e-06
k = 2, review 65, coalition [269]: local jShapley = -0.0001692694868687363
k = 2, review 65, coalition [1003]: local jShapley = -3.278217412145523e-05
k = 2, review 65, coalition [455]: local jShapley = 2.969776067174184e-05
k = 2, review 65, coalition [848]: local jShapley = -0.0009338354923626792
k = 2, review

k = 2, review 72, coalition [209]: local jShapley = 5.028486143543915e-05
k = 2, review 72, coalition [978]: local jShapley = -0.0001644315173630386
k = 2, review 72, coalition [101]: local jShapley = -0.00034431939846627007
k = 2, review 72, coalition [434]: local jShapley = -3.9445272420635006e-05
k = 2, review 72, coalition [3]: local jShapley = -0.0001244876518328922
k = 2, review 72, coalition [174]: local jShapley = 1.038123191600381e-05
k = 2, review 72, coalition [269]: local jShapley = -0.0003256419630302008
k = 2, review 72, coalition [1003]: local jShapley = -2.833395540503487e-05
k = 2, review 72, coalition [455]: local jShapley = 0.0006798494707373014
k = 2, review 72, coalition [848]: local jShapley = -0.0007475157131050131
k = 2, review 73, coalition [209, 978]: local jShapley = 3.021799953171849e-07
k = 2, review 73, coalition [101, 434]: local jShapley = -4.562246048509391e-06
k = 2, review 73, coalition [3, 174]: local jShapley = 7.704610184091877e-06
k = 2, review 73

k = 2, review 79, coalition [434]: local jShapley = -4.804160755598645e-07
k = 2, review 79, coalition [3]: local jShapley = -5.695626999735514e-06
k = 2, review 79, coalition [174]: local jShapley = 2.0420058233927653e-05
k = 2, review 79, coalition [269]: local jShapley = -0.0003154676227021868
k = 2, review 79, coalition [1003]: local jShapley = -3.608071863794481e-05
k = 2, review 79, coalition [455]: local jShapley = 0.00010553694654644614
k = 2, review 79, coalition [848]: local jShapley = -0.00038027604386276073
k = 2, review 80, coalition [209, 978]: local jShapley = 2.2248534064878607e-06
k = 2, review 80, coalition [101, 434]: local jShapley = -3.120918397664563e-05
k = 2, review 80, coalition [3, 174]: local jShapley = 3.486602191731186e-06
k = 2, review 80, coalition [269, 1003]: local jShapley = -4.414209109455613e-05
k = 2, review 80, coalition [455, 848]: local jShapley = 1.256494433333274e-05
k = 2, review 80, coalition [209]: local jShapley = 2.783590467476233e-05
k = 

k = 2, review 86, coalition [1003]: local jShapley = -9.158901009880765e-05
k = 2, review 86, coalition [455]: local jShapley = 0.00012791395210352236
k = 2, review 86, coalition [848]: local jShapley = 0.0
k = 2, review 87, coalition [209, 978]: local jShapley = -6.731008603953712e-07
k = 2, review 87, coalition [101, 434]: local jShapley = -2.8008997683985914e-06
k = 2, review 87, coalition [3, 174]: local jShapley = -3.629148054995802e-06
k = 2, review 87, coalition [269, 1003]: local jShapley = -1.0065994212311892e-05
k = 2, review 87, coalition [455, 848]: local jShapley = 4.191293343525547e-06
k = 2, review 87, coalition [209]: local jShapley = 2.7978815607384363e-05
k = 2, review 87, coalition [978]: local jShapley = -8.875803034475192e-06
k = 2, review 87, coalition [101]: local jShapley = -0.00034310399189067575
k = 2, review 87, coalition [434]: local jShapley = -9.717431689483016e-06
k = 2, review 87, coalition [3]: local jShapley = -1.932934463077557e-05
k = 2, review 87, c

k = 2, review 94, coalition [101, 434]: local jShapley = -7.88189710945586e-06
k = 2, review 94, coalition [3, 174]: local jShapley = 1.9953855687665343e-07
k = 2, review 94, coalition [269, 1003]: local jShapley = -3.752161764434351e-05
k = 2, review 94, coalition [455, 848]: local jShapley = -4.22568456480555e-05
k = 2, review 94, coalition [209]: local jShapley = 2.9208284064375777e-05
k = 2, review 94, coalition [978]: local jShapley = -1.4415978416654724e-05
k = 2, review 94, coalition [101]: local jShapley = -5.430472848535047e-06
k = 2, review 94, coalition [434]: local jShapley = -2.0300085530865007e-06
k = 2, review 94, coalition [3]: local jShapley = -4.48165985309633e-05
k = 2, review 94, coalition [174]: local jShapley = 5.499064499159707e-06
k = 2, review 94, coalition [269]: local jShapley = -0.0001946488480278279
k = 2, review 94, coalition [1003]: local jShapley = -5.534887648417189e-05
k = 2, review 94, coalition [455]: local jShapley = 8.459788296815847e-05
k = 2, rev

#### Also obtain Joint Shapley values for the target vocab

In [30]:
target_vocab_DSA_scores = {}
for key, ss_reviews in target_vocab_DSA.items():
    scores = []
    for review in ss_to_vc(ss_reviews, DSA_vectorizer):
        try:
            score = calculate_phi_t(
                sample_review_oh,
                value_f,
                review,
                num_DSA_features,
                q_values,
                num_sampling_iterations
            )
        except:
            score = np.nan
        scores.append(score)
    target_vocab_DSA_scores[key] = scores

In [31]:
target_vocab_DSA_scores

{'t2dsa': [-8.450741946849841e-07,
  3.116703822067041e-05,
  -2.1688758331398024e-07,
  -4.523688107136829e-05,
  3.536020269738501e-05]}

#### Obtain local Joint Shapleys for the training set

In [32]:
local_js = []
for review_vc in ss_to_vc(test_reviews_ss, DSA_vectorizer): 
    local_js.append(
        calculate_phi_t(
            sample_review_oh,
            value_f,
            review_vc,
            num_DSA_features,
            q_values,
            num_sampling_iterations
        )
    )

In [33]:
local_js

[-7.402459153879807e-07,
 -1.3678949492307517e-06,
 -1.1080664913381875e-05,
 -1.1236244416533324e-05,
 4.330645862204569e-07,
 -1.228861955317951e-07,
 1.0681923121222902e-06,
 -1.160797778545357e-05,
 -6.050747945754944e-07,
 -2.5867743711784743e-06,
 2.2655398551014878e-07,
 -7.888861042691908e-07,
 -3.972665344648958e-06,
 1.2370675424105868e-05,
 -1.3575481095945278e-06,
 -7.573600240587811e-07,
 -2.5901421454655703e-06,
 1.35255635757637e-06,
 -2.261989164966636e-06,
 1.0947966381797353e-06,
 1.4659197825819656e-06,
 -1.0957644118733495e-06,
 9.349390021555675e-09,
 1.6133197548211296e-06,
 -8.613295497304406e-05,
 -6.110647657631004e-06,
 3.335423390944894e-07,
 -3.68579690503407e-07,
 3.31136159233496e-06,
 -2.6601624971077263e-06,
 -1.4803998726295828e-05,
 1.296867650203927e-07,
 -2.799673424346092e-07,
 8.160561470832455e-07,
 -4.560203744823152e-07,
 -1.0000839863972077e-06,
 -2.788643918913825e-06,
 -8.939362180664342e-06,
 6.9136950065141095e-06,
 -2.330155943918116e-06,
